In [ ]:
import sqlite3
from datetime import datetime, timedelta

In [ ]:
#connect to local sqlite database (create if it is not existing)
conn = sqlite3.connect('office_data.db')
cursor = conn.cursor()

In [ ]:
#Create an attendance table
cursor.execute("""
create table if not exists attendance(
  id integer primary key autoincrement,
  name text,
  date text
)
""")

In [ ]:
#Insert some sample data

today = datetime.now()

for i in range(3):
  day = today - timedelta(days=i)
  for j in range(100 + i * 10):
    cursor.execute("insert into attendance(name,date) values (?,?)", (f"person_{j+1}" , day.strftime("%Y-%m-%d")))

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('office_data.db')
cursor = conn.cursor()

cursor.execute("select * from attendance limit 5")
rows = cursor.fetchall()
for row in rows:
  print(row)

conn.commit()
conn.close()

(1, 'person_1', '2025-07-21')
(2, 'person_2', '2025-07-21')
(3, 'person_3', '2025-07-21')
(4, 'person_4', '2025-07-21')
(5, 'person_5', '2025-07-21')


In [ ]:
def get_attendance_count(date_str):
  conn = sqlite3.connect('office_data.db')
  cursor = conn.cursor()

  cursor.execute("select count(*) from attendance where date = ?", (date_str,))
  count = cursor.fetchone()[0]

  conn.commit()
  conn.close()

  return count

In [ ]:
get_attendance_count('2025-07-20')

110

**Using Natural languages**

In [ ]:
def handle_query(user_query):
  if 'yesterday' in user_query.lower():
    date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    count = get_attendance_count(date)
    return f"Bot: Yesterday, {count} people were in the office"
  else:
    return "Bot: Sorry, I can only answer about yesterday attendance right now"

In [ ]:
user_query = "How many people were in the office yesterday?"
response = handle_query(user_query)
print(response)

Bot: Yesterday, 110 people were in the office


**Bot that can answer attendance for any day**

In [ ]:
import re
re.search(r"\d{4}-\d{2}-\d{2}" , "How many people were in the office on 2025-07-19")

<re.Match object; span=(38, 48), match='2025-07-19'>

In [ ]:
re.search(r"\d{4}-\d{2}-\d{2}" , "How many people were in the office on 2025-07-19").group()

'2025-07-19'

In [ ]:
#Function to extract the date from query
def extract_date_from_query(user_query):
  today = datetime.now().date()
  if "today" in user_query.lower():
    return today.strftime("%Y-%m-%d")
  elif "yesterday" in user_query.lower():
    yesterday = today - timedelta(days=1)
    return yesterday.strftime("%Y-%m-%d")
  else:
    #match the given date
    date_match = re.search(r"\d{4}-\d{2}-\d{2}" , user_query)
    if date_match:
      return date_match.group(0)
    return None

In [ ]:
datetime.now().date()

datetime.date(2025, 7, 21)

In [ ]:
extract_date_from_query('How many people were in the office on 2025-07-19')

'2025-07-19'

In [ ]:
#Chatbot query handler
def chatbot_query_handler(user_query):
  conn = sqlite3.connect('office_data.db')
  cursor = conn.cursor()

  target_date = extract_date_from_query(user_query)
  if not target_date:
    return """Bot: Sorry I couldn't find the date in your querry.
    Please try again using a valid date or words like 'today' or 'yesterday'"""

  cursor.execute("select count(*) from attendance where date = ?", (target_date,))
  count = cursor.fetchone()[0]

  conn.commit()
  conn.close()

  if not count:
    return f"Bot: No attendance record found for {target_date}"
  else:
    return f"Bot: on {target_date}, {count} people were in the office"

In [ ]:
querries = [
    "How many people were in the office today?",
    "How many people were in the office on 2025-07-19?",
    "How many people were in the office yesterday?",
    "Show me the attendance for 2025-07-18",
    "Show me the attendance for 2025-07-20",
    "What is attendance?"
]

for query in querries:
  print(f"User: {query}")
  response = chatbot_query_handler(query)
  print(f"{response}")
  print()

User: How many people were in the office today?
Bot: on 2025-07-21, 100 people were in the office

User: How many people were in the office on 2025-07-19?
Bot: on 2025-07-19, 120 people were in the office

User: How many people were in the office yesterday?
Bot: on 2025-07-20, 110 people were in the office

User: Show me the attendance for 2025-07-18
Bot: No attendance record found for 2025-07-18

User: Show me the attendance for 2025-07-20
Bot: on 2025-07-20, 110 people were in the office

User: What is attendance?
Bot: Sorry I couldn't find the date in your querry. 
    Please try again using a valid date or words like 'today' or 'yesterday'



In [ ]:
user_input = input("What do you want to know? ")

response = chatbot_query_handler(user_input)
print(f"{response}")

What do you want to know? attendance cout of 2025-07-20
Bot: on 2025-07-20, 110 people were in the office


In [ ]:
queries = ["Tell me about those employee who present yesterday but not today",
           "Difference in attendance count of last 3 days"]